# Mott-Schottky model for calculating the space charge potential

Analysis of grain boundary effects on ionic conductivity typically considers the space-charge potential as a characteristic parameter. However $r_{GB}$ can be obtained directly from experimental impedance spectroscopy, whereas the space charge potential cannot and is therefore approximated using the Mott-Schottky model. 

$$ r_{GB} = \frac{\rho_{i,x}}{\rho_{i,\infty}} = \frac{\mathrm{exp}(z\Phi_0 / k_BT)}{2z \Phi_0 / k_BT } $$

The conventional Mott-Schottky analysis assumes that the grain boundart region is negligably thin and that oxygen vacancies are fully depleted in the space-charge region. This gives an analytical description of the space-charge behaviour which can be completely characterised from the space charge potential. 

By calculating the grain boundary resistivity of the system, the Mott-Schotty model can be used to calculate the space charge potential allowing a comparison between the value calculated using the Poisson-Boltzmann solver and the value calculated using the Mott-Schottky model. 

Using ```sympy.solvers.solve ``` the function required to solve the Mott-Scottky model for $\Phi$ was calculated. 

```
x = Symbol('x')
y = Symbol('y')

print( solve (exp(x) / ( 2 * x ) - y, x )  )


[-LambertW(-1/(2*y))]
```

A function named ```solve_MS_for_phi``` was created in the source code which takes the input ```y```, the grain boundary resistivity, and returns ``` -mpmath.lambertw(-1/(2*y),k=-1) ```. The output for this is then multiplied by the Boltzmann constant, and the temperature, and divided by the charge to give the space charge potential.

# The Debye length and space charge width

The Poisson-Boltzmann equation can be manipulated to give the Debye length. The Debye length is the "characteristic length scale" from the Debye-Huckel equation, which is the linearised Poisson-Boltzmann equation used for dilute systems.

$$ L_D = \left( \frac{\epsilon \epsilon_0 k_BT}{2 F^2 c_{i, \infty } } \right) ^ \frac{1}{2} $$

Where $L_D$ is the Debye length, $\epsilon$ is the dielectric, $\epsilon_0$ is the vacuum permittivity, $k_B$ is the Boltzmann constant, $T$ is the temperature, $F$ is Faraday's constant and $c_{i,\infty}$ is the bulk defect density. 
$L_D$ is then used to calculate the width of the space charge region.

$$ \delta_{sc} = 2L_D \left(\frac{\Phi_0 z}   {k_BT} \right)^ \frac{1}{2}$$

Where $\Phi_0$ is the space charge potential. The space charge potential can be calculated from the Poisson-Boltzmann equation or from the Mott-Schottky model to find their respective space charge widths. 

In [1]:
import os
os.getcwd()

import sys
sys.path.append('/Users/glw33/source/project')

from project.defect_species import Defect_Species
from project.set_of_sites import Set_of_Sites
from project.calculation import Calculation
from project.constants import boltzmann_eV
from project.onedefect_onedopant_calculations import *
from project.general_calculations import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

In [2]:
alpha = 0.0005

conv = 1e-8
grid_x_min = -2.5e-8
grid_x_max = +2.5e-8

limits = [grid_x_min-5e-10, grid_x_max+5e-10]

index = 111

valence = [ +1.0, -1.0 ]
site_labels = ['site_1', 'site_2']
defect_labels = ['defect_1', 'defect_2']

dielectric = 50
len_b = 1e-9
len_c = 1e-9

temp = [ 773.15 ]
mole_fractions = [0.2, 0.2]

desired_mobile_defect_mf = 0.2
percentage_dopant = 20

data_2 = '/Users/glw33/Documents/papers/Gd-CeO2_SC/supporting_information/examples/data_for_examples/example_data_4_alternate_sites_one_defect.txt' 

labels=[ 'temp', 'r_gb', 'max phi', 'MS phi', 'Debye length', 'space charge width' ]
data = pd.DataFrame( columns=labels )

boundary_conditions = 'periodic'

defect_species = { l : Defect_Species( l, v, m ) for l, v, m in zip( defect_labels, valence, mole_fractions ) }

all_sites = Set_of_Sites.set_of_sites_from_input_data( data_2, limits, defect_species )
for site in all_sites.subset( 'site_2' ):
    site.defect_with_label('defect_2').fixed = True

grid = Grid.grid_from_set_of_sites( all_sites, limits[0], limits[1], len_b, len_c )

for t in temp:

    c = Calculation( grid, alpha, conv, t, boundary_conditions )
    
    c.solve()
    c.form_subgrids( site_labels )
    c.mole_fractions
    c.calculate_resistivity_ratio( desired_mobile_defect_mf )
    c.solve_MS_approx_for_phi( valence[0] )
    c.calculate_debye_length()
    c.calculate_space_charge_width( valence[0] )
    
    data = data.append( pd.DataFrame( [[ t, c.resistivity_ratio, max( c.phi ), c.ms_phi, 
                               c.debye_length, c.space_charge_width ]], columns=labels ) )
    
# Ea = calculate_activation_energy( resistivity_ratio_list, temp ) 

In [3]:
data

,temp,r_gb,max phi,MS phi,Debye length,space charge width
0,773.15,4.430889,0.820338,0.227045647768514,2.177104,15.278732


The space-charge potential calculated using the Mott-Schottky model can be compared to the space-charge potential calculated using the Poisson-Boltzmann solver as demonstrated below.

In [4]:
plt.plot( temp, space_charge_potential_list, label = '$\Phi_0^{PB}$' )
plt.plot( temp, ms_space_charge_potential_list, label = ' $\Phi_0^{MS}' )
plt.xlabel( 'Temperature (K)' )
plt.ylabel( 'Potential (eV)' )
plt.show()

NameError: name 'space_charge_potential_list' is not defined

In [ ]:
plt.plot( temp, space_charge_width_list, label = 'space charge width' )
plt.xlabel( 'Temperature (K)' )
plt.ylabel( 'space charge width' )
plt.show()